In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm  # Thư viện tuyệt vời cho các backbone
import cv2  # OpenCV để đọc ảnh
from tqdm import tqdm # Thanh tiến trình
import matplotlib.pyplot as plt

# --- LỚP CẤU HÌNH TRUNG TÂM ---
# Quản lý tất cả các siêu tham số (hyperparameters) tại đây
class CFG:
    # --- Đường dẫn (Paths) ---
    # (Hãy điều chỉnh các đường dẫn này cho đúng với môi trường của bạn)
    BASE_PATH = '/kaggle/input/csiro-biomass'
    TRAIN_CSV = os.path.join(BASE_PATH, 'train.csv')
    IMAGE_DIR = os.path.join(BASE_PATH, 'train')
    
    # --- Cài đặt Mô hình ---
    MODEL_NAME = 'coatnet_rmlp_2_rw_384.sw_in12k_ft_in1k' # Bạn có thể đổi sang 'resnet50'
    PRETRAINED = True
    IMG_SIZE = 384 # Kích thước ảnh đầu vào (từ 1000x1000 nén xuống)
    SEED = 42
    # --- Cài đặt Huấn luyện ---
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 2   # Bắt đầu với số nhỏ (vì 2 luồng ảnh 768x768 sẽ tốn VRAM)
    EPOCHS = 20
    LEARNING_RATE = 1e-4      # LR cho Giai đoạn 1 (huấn luyện heads)
    
    # --- Cài đặt Giai đoạn 2 (Fine-tuning) ---
    FREEZE_EPOCHS = 5         # Số epochs chỉ huấn luyện 'heads'
    FINETUNE_LR = 1e-5        # LR thấp hơn cho Giai đoạn 2 (toàn bộ mô hình)
    
    NUM_WORKERS = 2  # Số luồng tải dữ liệu
    
    # --- Mục tiêu & Loss ---
    TARGET_COLS = ['Dry_Total_g', 'GDM_g', 'Dry_Green_g']
    LOSS_WEIGHTS = {
        'total_loss': 0.5,
        'gdm_loss': 0.2,
        'green_loss': 0.1
    }
    # 5 MỤC TIÊU ĐỂ TÍNH ĐIỂM (theo đúng thứ tự của cuộc thi)
    ALL_TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
    
    # Trọng số R2 (theo thứ tự của ALL_TARGET_COLS)
    R2_WEIGHTS = [0.1, 0.1, 0.1, 0.2, 0.5]

# --- In ra để xác nhận ---
print(f"Sử dụng thiết bị: {CFG.DEVICE}")
print(f"Backbone mô hình: {CFG.MODEL_NAME}")
print(f"Kích thước ảnh huấn luyện: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")

In [ ]:
import random
def set_seed(seed: int = 42):
    """
    Ensures full reproducibility across:
    - Python's random module
    - NumPy operations
    - PyTorch (CPU & GPU)
    - CUDA convolution algorithms

    Args:
        seed (int): The random seed to set (default = 42)
    """
    # Python
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

    # NumPy
    np.random.seed(seed)

    # PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Ensures deterministic behavior
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    print(f"✅ Random seed set to {seed} for full reproducibility.")

set_seed(CFG.SEED)
oof_dfs = []


In [ ]:
# --- Load data and perform pivot ---

print(f"Loading {CFG.TRAIN_CSV}...")
try:
    # 1. Read the original CSV file (in long format)
    df_long = pd.read_csv(CFG.TRAIN_CSV)
    print(f"The original 'long' table has {len(df_long)} rows.")
    
    # 2. Perform pivot
    # Use 'image_path' as the unique index
    # Convert 'target_name' values into new columns
    # Use values from the 'target' column
    df_wide = df_long.pivot(
        index='image_path',
        columns='target_name',
        values='target'
    )
    
    # 3. Clean up the DataFrame
    # After pivoting, 'image_path' becomes the index.
    # Use reset_index() to turn it back into a normal column.
    df_wide = df_wide.reset_index()
    df_wide.columns.name = None  # Remove the 'target_name' label from column axis
    
    print(f"Pivot complete! The new 'wide' table has {len(df_wide)} rows (one per image).")
    
    # 4. Display the first 5 rows of the new wide table
    print("\n--- First 5 rows of df_wide ---")
    print(df_wide.head())
    
    # 5. (Optional) Check if our target columns are present
    print("\n--- Columns present in df_wide ---")
    print(df_wide.columns.tolist())
    
except FileNotFoundError:
    print(f"ERROR: File not found at {CFG.TRAIN_CSV}")
    print("Please verify CFG.TRAIN_CSV path.")
    # Create an empty placeholder df_wide so later steps don’t fail
    df_wide = pd.DataFrame(columns=['image_path'] + CFG.TARGET_COLS)


In [ ]:
# from albumentations import (
#     Compose, 
#     Resize, 
#     Normalize,
#     HorizontalFlip, 
#     VerticalFlip,
#     RandomRotate90,  # Only rotates by 90, 180, or 270 degrees
#     ColorJitter
# )

# # --- Define Data Augmentation Pipelines ---

# def get_train_transforms():
#     """
#     Data augmentation pipeline for training.
#     Applied independently to img_left and img_right.
#     """
#     return Compose([
#         # 1. Geometric augmentations
#         HorizontalFlip(p=0.5),
#         VerticalFlip(p=0.5),
#         RandomRotate90(p=0.5),  # Randomly rotate by 90, 180, or 270 degrees

#         # 2. Photometric (color-based) augmentations
#         ColorJitter(
#             brightness=0.2, 
#             contrast=0.2, 
#             saturation=0.2, 
#             hue=0.1, 
#             p=0.75
#         ),

#         # 3. Normalization (standard ImageNet mean/std)
#         Normalize(
#             mean=[0.485, 0.456, 0.406],
#             std=[0.229, 0.224, 0.225]
#         ),

#         # 4. Resize and convert to tensor
#         # (Resize after augmentations to retain visual quality)
#         Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
#         ToTensorV2()
#     ])


from albumentations import (
    Compose, 
    Resize, 
    Normalize,
    HorizontalFlip, 
    VerticalFlip,
    RandomRotate90, 
    ColorJitter,
    RandomBrightnessContrast,
    ShiftScaleRotate,
    Blur,
    GaussNoise,
    OpticalDistortion,
    GridDistortion,
    ElasticTransform,
    CoarseDropout,
    OneOf
)
from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    """
    Enhanced data augmentation pipeline for training.
    Includes geometric, color, blur/noise, and distortion-based augmentations.
    Applied independently to each image (e.g., img_left and img_right).
    """
    return Compose([
        # 1. Geometric augmentations
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        RandomRotate90(p=0.5),
        ShiftScaleRotate(
            shift_limit=0.05,  # ±5% shift
            scale_limit=0.1,   # ±10% scale
            rotate_limit=15,   # ±15° rotation
            border_mode=0,
            p=0.6
        ),

        # 2. Distortion augmentations (simulate lens / perspective variations)
        OneOf([
            OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
            GridDistortion(num_steps=5, distort_limit=0.03, p=1.0),
            ElasticTransform(alpha=30, sigma=5, alpha_affine=10, p=1.0)
        ], p=0.3),

        # 3. Photometric augmentations
        ColorJitter(
            brightness=0.2, 
            contrast=0.2, 
            saturation=0.2, 
            hue=0.1, 
            p=0.75
        ),
        RandomBrightnessContrast(
            brightness_limit=0.2, 
            contrast_limit=0.2, 
            p=0.5
        ),

        # 4. Blur and noise augmentations
        OneOf([
            Blur(blur_limit=3, p=1.0),
            GaussNoise(var_limit=(10.0, 50.0), p=1.0)
        ], p=0.3),

        # 5. Random occlusion
        CoarseDropout(
            max_holes=8,
            max_height=CFG.IMG_SIZE // 10,
            max_width=CFG.IMG_SIZE // 10,
            fill_value=0,
            p=0.4
        ),

        # 6. Normalize and resize (last to preserve augmentations)
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ])


def get_valid_transforms():
    """
    Validation pipeline — only normalization and resizing (no augmentations).
    """
    return Compose([
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        ToTensorV2()
    ])

# --- Print to confirm ---
print("Augmentation functions defined successfully.")
print(f"Training images will be augmented and resized to: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")
print(f"Validation images will only be resized to: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")


In [ ]:
# (This is the NEW version of the BiomassDataset class)

class BiomassDataset(Dataset):
    """
    Custom dataset for the 'Two-Stream' strategy.

    Returns:
        (img_left, img_right, train_targets (3), all_targets (5))
    """
    def __init__(self, df, transforms_fn, image_dir, train_target_cols, all_target_cols):
        self.df = df
        self.transforms_fn = transforms_fn
        self.image_dir = image_dir
        
        # Pre-store for faster access
        self.image_paths = df['image_path'].values
        # 3 targets used for training loss computation
        self.train_targets = df[train_target_cols].values
        # 5 targets used for R² evaluation
        self.all_targets = df[all_target_cols].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. Retrieve paths and target values
        img_path_suffix = self.image_paths[idx]
        train_target_vals = self.train_targets[idx]
        all_target_vals = self.all_targets[idx]
        
        # 2. Read the original image (2000x1000)
        filename = os.path.basename(img_path_suffix)
        full_path = os.path.join(self.image_dir, filename)
        image = cv2.imread(full_path)
        if image is None:
            raise FileNotFoundError(f"Cannot read image: {full_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 3. Crop into two halves (Left and Right)
        height, width, _ = image.shape
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]
        
        # 4. Apply augmentations independently
        transforms = self.transforms_fn()
        img_left_tensor = transforms(image=img_left)['image']
        
        # Use a NEW augmentation pipeline for the right image
        transforms_2 = self.transforms_fn()
        img_right_tensor = transforms_2(image=img_right)['image']
        
        # 5. Convert target values to tensors
        train_target_tensor = torch.tensor(train_target_vals, dtype=torch.float32)
        all_targets_tensor = torch.tensor(all_target_vals, dtype=torch.float32)
        
        # 6. Return tuple of tensors
        return img_left_tensor, img_right_tensor, train_target_tensor, all_targets_tensor


In [ ]:
# --- Test Dataset and DataLoader ---

print("\nTesting Dataset & DataLoader...")

try:
    # 1. Create a small dataset (using training augmentations)
    # Take the first 10 rows from df_wide for testing
    test_df_subset = df_wide.head(10)
    
    train_dataset = BiomassDataset(
        df=test_df_subset,
        transforms_fn=get_train_transforms,  # Pass the function (not called yet)
        image_dir=CFG.IMAGE_DIR,
        train_target_cols=CFG.TARGET_COLS,
        all_target_cols=CFG.ALL_TARGET_COLS
    )
    
    # 2. Create a DataLoader
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.BATCH_SIZE,
        shuffle=True,
        num_workers=CFG.NUM_WORKERS
    )
    
    # 3. Load one batch
    print(f"Loading 1 batch (batch_size={CFG.BATCH_SIZE})...")
    img_left_batch, img_right_batch, train_targets_batch, all_targets_batch = next(iter(train_loader))
    
    # 4. Print output shapes
    print("\n--- Output Batch Shapes ---")
    print(f"  Left images:  {img_left_batch.shape}")
    print(f"  Right images: {img_right_batch.shape}")
    print(f"  Train targets: {train_targets_batch.shape}")
    print(f"  All targets:   {all_targets_batch.shape}")
    
    print("\n--- Expected Shapes ---")
    print(f"  Images:   [Batch, Channels, Height, Width] -> [{CFG.BATCH_SIZE}, 3, {CFG.IMG_SIZE}, {CFG.IMG_SIZE}]")
    print(f"  Targets:  [Batch, Num_Train_Targets] -> [{CFG.BATCH_SIZE}, {len(CFG.TARGET_COLS)}]")
    
    # (Optional) Display one sample image
    img = img_left_batch[0].permute(1, 2, 0).cpu().numpy()
    img = img * 0.229 + 0.485  # Undo normalization (approx.)
    plt.imshow(np.clip(img, 0, 1))
    plt.title("Sample Image from Batch (Left)")
    plt.axis('off')
    plt.show()

except Exception as e:
    print(f"\nERROR while testing DataLoader: {e}")
    print("Please check CFG paths and image directory structure.")


In [ ]:
class BiomassModel(nn.Module):
    """
    Two-Stream, Three-Head Specialized Model.
    
    Architecture:
    1. A shared backbone (e.g., ConvNeXt) processes both left and right image halves.
    2. The same backbone is applied independently to img_left and img_right.
    3. The two resulting feature vectors are concatenated.
    4. The combined feature vector is fed into three separate MLP heads:
       - Head 1 → Dry_Total_g
       - Head 2 → GDM_g
       - Head 3 → Dry_Green_g
    """
    def __init__(self, model_name, pretrained=True, n_targets=3):
        super(BiomassModel, self).__init__()
        
        # 1. Load the shared backbone
        # num_classes=0 removes the original classification layer
        # global_pool='avg' adds global average pooling to produce a 1D feature vector
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,
            global_pool='avg'
        )
        
        # 2. Get the feature dimension from the backbone
        # Example: convnext_tiny -> 768, resnet50 -> 2048
        self.n_features = self.backbone.num_features
        
        # 3. Compute the combined feature dimension after fusion
        # (features_left + features_right)
        self.n_combined_features = self.n_features * 2
        
        # 4. Define three independent MLP heads (one for each target)
        
        # --- Head for Dry_Total_g ---
        self.head_total = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)  # Single output value
        )
        
        # --- Head for GDM_g ---
        self.head_gdm = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )
        
        # --- Head for Dry_Green_g ---
        self.head_green = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1)
        )

    def forward(self, img_left, img_right):
        # 1. Forward pass through stream 1 (Left image)
        features_left = self.backbone(img_left)   # Shape: [batch, n_features]
        
        # 2. Forward pass through stream 2 (Right image)
        features_right = self.backbone(img_right) # Shape: [batch, n_features]
        
        # 3. Feature fusion (concatenate left and right)
        combined = torch.cat([features_left, features_right], dim=1)  # [batch, n_combined_features]
        
        # 4. Pass through three specialized MLP heads
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)
        
        # 5. Return three outputs (as separate tensors for weighted loss computation)
        return out_total, out_gdm, out_green


In [ ]:
# --- Test Model Architecture ---
print("\nTesting model architecture...")

try:
    # 1. Initialize the model
    model = BiomassModel(
        model_name=CFG.MODEL_NAME,
        pretrained=False  # No need to load pretrained weights for architecture check
    ).to(CFG.DEVICE)
    
    # 2. Create dummy input data
    # (Batch size, Channels, Height, Width)
    dummy_left = torch.randn(CFG.BATCH_SIZE, 3, CFG.IMG_SIZE, CFG.IMG_SIZE).to(CFG.DEVICE)
    dummy_right = torch.randn(CFG.BATCH_SIZE, 3, CFG.IMG_SIZE, CFG.IMG_SIZE).to(CFG.DEVICE)
    
    print(f"Feeding two dummy batches of shape {dummy_left.shape} into the model...")
    
    # 3. Run a forward pass
    out_total, out_gdm, out_green = model(dummy_left, dummy_right)
    
    # 4. Print output shapes
    print("\n--- Model Output Shapes ---")
    print(f"  Output (Total): {out_total.shape}")
    print(f"  Output (GDM):   {out_gdm.shape}")
    print(f"  Output (Green): {out_green.shape}")
    
    print("\n--- Expected Shapes for Each Output ---")
    print(f"  Expected: [Batch, 1] -> [{CFG.BATCH_SIZE}, 1]")

except Exception as e:
    print(f"\nERROR while testing model: {e}")


In [ ]:
class WeightedBiomassLoss(nn.Module):
    """
    Weighted composite loss function for biomass prediction.

    Calculates three separate SmoothL1 losses (for Total, GDM, and Green)
    and combines them using the predefined weights from CFG.
    """
    def __init__(self, loss_weights_dict):
        super(WeightedBiomassLoss, self).__init__()
        
        # Use a single SmoothL1 (Huber) loss function for stability
        self.criterion = nn.SmoothL1Loss()
        
        # Store weight dictionary (e.g., {'total_loss': 0.5, 'gdm_loss': 0.2, 'green_loss': 0.1})
        self.weights = loss_weights_dict

    def forward(self, predictions, targets):
        """
        Args:
            predictions: tuple of (out_total, out_gdm, out_green) from the model
            targets: tensor of shape [batch_size, 3] containing ground truth values
                     Columns correspond to:
                       0 - Dry_Total_g
                       1 - GDM_g
                       2 - Dry_Green_g
        Returns:
            Weighted total loss (scalar)
        """
        # 1. Unpack predictions
        pred_total, pred_gdm, pred_green = predictions
        
        # 2. Extract true target components
        true_total = targets[:, 0].unsqueeze(-1)  # [batch, 1]
        true_gdm   = targets[:, 1].unsqueeze(-1)  # [batch, 1]
        true_green = targets[:, 2].unsqueeze(-1)  # [batch, 1]
        
        # 3. Compute individual losses
        loss_total = self.criterion(pred_total, true_total)
        loss_gdm   = self.criterion(pred_gdm, true_gdm)
        loss_green = self.criterion(pred_green, true_green)
        
        # 4. Compute weighted sum of all three losses
        total_loss = (
            self.weights['total_loss'] * loss_total +
            self.weights['gdm_loss'] * loss_gdm +
            self.weights['green_loss'] * loss_green
        )
        
        return total_loss


In [ ]:
# --- Initialize and Test the Loss Function ---

print("\nInitializing loss function...")

# 1. Initialize
criterion = WeightedBiomassLoss(loss_weights_dict=CFG.LOSS_WEIGHTS)
criterion.to(CFG.DEVICE)

print(f"Loss function created with weights: {CFG.LOSS_WEIGHTS}")

# 2. Test the loss function
print("\nTesting loss function...")
try:
    # Create dummy prediction data (simulating model outputs)
    dummy_preds = (
        torch.randn(CFG.BATCH_SIZE, 1).to(CFG.DEVICE),  # out_total
        torch.randn(CFG.BATCH_SIZE, 1).to(CFG.DEVICE),  # out_gdm
        torch.randn(CFG.BATCH_SIZE, 1).to(CFG.DEVICE)   # out_green
    )
    
    # Create dummy target data (simulating dataloader outputs)
    dummy_targets = torch.randn(CFG.BATCH_SIZE, len(CFG.TARGET_COLS)).to(CFG.DEVICE)
    
    # Compute loss
    loss_value = criterion(dummy_preds, dummy_targets)
    
    print(f"  Dummy data: {CFG.BATCH_SIZE} samples")
    print(f"  Computed loss value: {loss_value.item():.4f}")
    print("✓ Loss function works correctly.")
    
except Exception as e:
    print(f"\nERROR while testing loss function: {e}")


In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

# --- Add to the CFG class ---
CFG.N_FOLDS = 5          # Use 5-Fold Cross-Validation
CFG.RANDOM_STATE = 42     # For reproducibility

print(f"\nPreparing {CFG.N_FOLDS}-Fold Cross-Validation...")

# 1. Create a new column 'fold' initialized to -1
df_wide['fold'] = -1

# 2. Create bins (groups) for the most important target: 'Dry_Total_g'
# We use pd.cut to divide continuous values into discrete bins.
# This helps StratifiedKFold maintain balanced distributions across folds.

num_bins = int(np.floor(1 + np.log2(len(df_wide))))  # Sturges' rule
if len(df_wide) > 100:
    num_bins = 10  # Use 10 bins if dataset is large enough

print(f"Using {num_bins} bins to stratify based on 'Dry_Total_g'.")

df_wide['total_bin'] = pd.cut(
    df_wide['Dry_Total_g'], 
    bins=num_bins, 
    labels=False  # Only numerical labels are needed
)

# 3. Initialize StratifiedKFold
# The split is based on the bin labels to preserve distribution of 'Dry_Total_g'
skf = StratifiedKFold(
    n_splits=CFG.N_FOLDS,
    shuffle=True,
    random_state=CFG.RANDOM_STATE
)

# 4. Assign fold numbers to each sample
# skf.split returns (train_indices, valid_indices)
# We only need valid_indices to assign fold IDs
for fold_num, (train_idx, valid_idx) in enumerate(skf.split(df_wide, df_wide['total_bin'])):
    df_wide.loc[valid_idx, 'fold'] = fold_num

# 5. Inspect the fold distribution
print("\n--- Sample Count per Fold ---")
print(df_wide['fold'].value_counts().sort_index())

print("\n--- df_wide after adding 'fold' column (first 5 rows) ---")
print(df_wide.head())


In [ ]:
from sklearn.metrics import r2_score

def calculate_competition_score(all_preds_3, all_targets_5):
    """
    Compute the weighted R² score used in the competition.
    
    Args:
        all_preds_3 (dict): Dictionary containing 3 NumPy arrays predicted by the model.
                            Keys: 'total', 'gdm', 'green'
        all_targets_5 (np.ndarray): Array of shape [N, 5] containing ground-truth values
                                    for all biomass components.
    
    Returns:
        float: Final weighted R² score.
    """
    
    # 1. Reconstruct all 5 predictions from the 3 model outputs
    pred_total = all_preds_3['total']
    pred_gdm = all_preds_3['gdm']
    pred_green = all_preds_3['green']
    
    # Derived predictions based on biological relationships
    pred_clover = np.maximum(0, pred_gdm - pred_green)
    pred_dead = np.maximum(0, pred_total - pred_gdm)
    
    # Combine all 5 predictions in the exact order of CFG.ALL_TARGET_COLS
    # ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
    y_preds = np.stack([
        pred_green,
        pred_dead,
        pred_clover,
        pred_gdm,
        pred_total
    ], axis=1)  # Shape: [N, 5]
    
    y_true = all_targets_5  # Shape: [N, 5]

    # 2. Compute R² for each target
    # sklearn's r2_score supports multi-output regression
    r2_scores = r2_score(y_true, y_preds, multioutput='raw_values')
    
    # 3. Compute weighted average R² using competition weights
    weighted_r2_total = 0.0
    for i, weight in enumerate(CFG.R2_WEIGHTS):
        weighted_r2_total += r2_scores[i] * weight
        
    return weighted_r2_total


In [ ]:
import torch.optim as optim
from tqdm import tqdm
import time
import gc

import torch
from tqdm import tqdm
import numpy as np

def train_one_epoch(model, loader, criterion, optimizer, device):
    """
    Standard FP32 training loop (no gradient accumulation, no mixed precision).
    """
    model.train()
    epoch_loss = 0.0
    pbar = tqdm(loader, desc="Training", leave=False)

    for step, (img_left, img_right, train_targets, _) in enumerate(pbar):
        img_left = img_left.to(device)
        img_right = img_right.to(device)
        targets = train_targets.to(device)

        optimizer.zero_grad()
        predictions = model(img_left, img_right)
        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    return epoch_loss / len(loader)


def validate_one_epoch(model, loader, criterion, device):
    model.eval()
    epoch_loss = 0.0
    
    all_preds_3 = {'total': [], 'gdm': [], 'green': []}
    all_targets_list = []
    all_paths = []

    with torch.no_grad():
        pbar = tqdm(loader, desc="Validating", leave=False)
        for batch in pbar:
            # --- Handle 4 or 5 elements from DataLoader ---
            if len(batch) == 5:
                img_left, img_right, train_targets, all_targets, image_paths = batch
            else:
                img_left, img_right, train_targets, all_targets = batch
                image_paths = [None] * len(img_left)  # placeholder

            img_left = img_left.to(device)
            img_right = img_right.to(device)
            train_targets = train_targets.to(device)

            # Forward pass
            pred_total, pred_gdm, pred_green = model(img_left, img_right)

            # Compute loss
            loss = criterion((pred_total, pred_gdm, pred_green), train_targets)
            epoch_loss += loss.item()

            # Collect predictions
            all_preds_3['total'].append(pred_total.cpu().numpy())
            all_preds_3['gdm'].append(pred_gdm.cpu().numpy())
            all_preds_3['green'].append(pred_green.cpu().numpy())
            all_targets_list.append(all_targets.cpu().numpy())
            all_paths.extend(image_paths)

    # Combine predictions
    preds_dict_np = {
        'total': np.concatenate(all_preds_3['total']).flatten(),
        'gdm': np.concatenate(all_preds_3['gdm']).flatten(),
        'green': np.concatenate(all_preds_3['green']).flatten()
    }
    targets_np_5 = np.concatenate(all_targets_list)
    competition_score = calculate_competition_score(preds_dict_np, targets_np_5)

    avg_epoch_loss = epoch_loss / len(loader)

    # if return_preds:
    #     # Align shapes for OOF saving
    #     val_preds = np.vstack([preds_dict_np['green'], preds_dict_np['gdm'], preds_dict_np['total']]).T
    #     val_trues = targets_np_5
    #     return avg_epoch_loss, competition_score, val_preds, val_trues, all_paths
    # else:
    return avg_epoch_loss, competition_score


In [ ]:
import time
import torch.optim as optim
import gc

import time, gc, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

import time
import torch
import torch.nn as nn
import torch.optim as optim
import gc
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader


def run_training(fold_to_run):
    """
    Main training function implementing 2-stage (Freeze/Unfreeze) strategy
    with FP16 mixed precision and gradient accumulation.
    """
    print(f"\n{'='*50}")
    print(f"🚀 STARTING TRAINING FOR FOLD {fold_to_run} 🚀")
    print(f"{'='*50}")
    
    start_time = time.time()

    # 1. Split data
    print(f"Splitting data for Fold {fold_to_run}...")
    train_df = df_wide[df_wide['fold'] != fold_to_run].reset_index(drop=True)
    valid_df = df_wide[df_wide['fold'] == fold_to_run].reset_index(drop=True)

    # 2. Create datasets and dataloaders
    train_dataset = BiomassDataset(
        df=train_df, transforms_fn=get_train_transforms, image_dir=CFG.IMAGE_DIR,
        train_target_cols=CFG.TARGET_COLS, all_target_cols=CFG.ALL_TARGET_COLS
    )
    valid_dataset = BiomassDataset(
        df=valid_df, transforms_fn=get_valid_transforms, image_dir=CFG.IMAGE_DIR,
        train_target_cols=CFG.TARGET_COLS, all_target_cols=CFG.ALL_TARGET_COLS
    )

    train_loader = DataLoader(
        train_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True,
        num_workers=CFG.NUM_WORKERS, pin_memory=True
    )
    valid_loader = DataLoader(
        valid_dataset, batch_size=CFG.BATCH_SIZE * 2, shuffle=False,
        num_workers=CFG.NUM_WORKERS, pin_memory=True
    )

    # 3. Initialize model, criterion, and optimizer
    print(f"Loading backbone '{CFG.MODEL_NAME}'...")
    model_base = BiomassModel(CFG.MODEL_NAME, CFG.PRETRAINED)

    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs with nn.DataParallel.")
        model = nn.DataParallel(model_base)
    else:
        model = model_base
    model.to(CFG.DEVICE)

    criterion = WeightedBiomassLoss(CFG.LOSS_WEIGHTS).to(CFG.DEVICE)

    # Initialize FP16 GradScaler

    # =================================================================
    # ✨ STAGE 1: Train "Heads" (Freeze Backbone)
    # =================================================================
    print(f"\n--- STAGE 1: Freezing Backbone (Training Heads Only) ---")
    print(f"Epochs: 1 to {CFG.FREEZE_EPOCHS} | LR: {CFG.LEARNING_RATE}")

    for param in model.module.backbone.parameters() if isinstance(model, nn.DataParallel) else model.backbone.parameters():
        param.requires_grad = False

    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=CFG.LEARNING_RATE
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

    best_score = -float('inf')

    for epoch in range(1, CFG.FREEZE_EPOCHS + 1):
        print(f"\n--- Epoch {epoch}/{CFG.EPOCHS} (Stage 1) ---")

        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, CFG.DEVICE)
        valid_loss, competition_score = validate_one_epoch(model, valid_loader, criterion, CFG.DEVICE)

        scheduler.step(valid_loss)

        print(f"Epoch {epoch} - Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f} | R² Score: {competition_score:.4f}")

        if competition_score > best_score:
            best_score = competition_score
            print(f"✨ R² improved! Saving model as 'best_model_fold{fold_to_run}.pth'...")
            torch.save(
                model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(),
                f'best_model_fold{fold_to_run}.pth'
            )

    # =================================================================
    # ✨ STAGE 2: Fine-tuning (Unfreeze Backbone)
    # =================================================================
    print(f"\n--- STAGE 2: Unfreezing Backbone (Fine-tuning) ---")
    print(f"Epochs: {CFG.FREEZE_EPOCHS + 1} to {CFG.EPOCHS} | LR: {CFG.FINETUNE_LR}")

    for param in model.module.backbone.parameters() if isinstance(model, nn.DataParallel) else model.backbone.parameters():
        param.requires_grad = True

    optimizer = optim.Adam(model.parameters(), lr=CFG.FINETUNE_LR)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3)

    for epoch in range(CFG.FREEZE_EPOCHS + 1, CFG.EPOCHS + 1):
        print(f"\n--- Epoch {epoch}/{CFG.EPOCHS} (Stage 2) ---")

        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, CFG.DEVICE)
        valid_loss, competition_score = validate_one_epoch(model, valid_loader, criterion, CFG.DEVICE)

        scheduler.step(valid_loss)

        print(f"Epoch {epoch} - Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f} | R² Score: {competition_score:.4f}")

        if competition_score > best_score:
            best_score = competition_score
            print(f"✨ R² improved! Saving model as 'best_model_fold{fold_to_run}.pth'...")
            torch.save(
                model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(),
                f'best_model_fold{fold_to_run}.pth'
            )

    # --- End of Training ---
    end_time = time.time()
    print(f"\n🎉 Finished Fold {fold_to_run} in {(end_time - start_time)/60:.2f} minutes.")
    print(f"Best R² Score: {best_score:.4f}")
        # ================================================================
    # 🧾 GENERATE OOF PREDICTIONS FOR THIS FOLD
    # ================================================================
    print(f"\nGenerating OOF predictions for Fold {fold_to_run}...")

    # Load the best saved model (highest R²)
    model_base = BiomassModel(CFG.MODEL_NAME, pretrained=False)
    model_path = f'best_model_fold{fold_to_run}.pth'
    state_dict = torch.load(model_path, map_location=CFG.DEVICE)
    model_base.load_state_dict(state_dict)
    model_base.to(CFG.DEVICE)
    model_base.eval()

    # Collect predictions
    preds_dict = {'total': [], 'gdm': [], 'green': []}
    all_targets_np = []
    image_paths = valid_df["image_path"].values

    with torch.no_grad():
        for (img_left, img_right, train_targets, all_targets) in tqdm(valid_loader, desc=f"OOF Inference Fold {fold_to_run}"):
            img_left = img_left.to(CFG.DEVICE)
            img_right = img_right.to(CFG.DEVICE)

            out_total, out_gdm, out_green = model_base(img_left, img_right)

            preds_dict['total'].append(out_total.cpu().numpy())
            preds_dict['gdm'].append(out_gdm.cpu().numpy())
            preds_dict['green'].append(out_green.cpu().numpy())
            all_targets_np.append(all_targets.cpu().numpy())

    # Concatenate all predictions
    preds_dict_np = {
        'total': np.concatenate(preds_dict['total']).flatten(),
        'gdm':   np.concatenate(preds_dict['gdm']).flatten(),
        'green': np.concatenate(preds_dict['green']).flatten()
    }
    all_targets_np = np.concatenate(all_targets_np)

    # Derive the remaining two predictions
    pred_clover = np.maximum(0, preds_dict_np['gdm'] - preds_dict_np['green'])
    pred_dead   = np.maximum(0, preds_dict_np['total'] - preds_dict_np['gdm'])

    # Create fold DataFrame
    fold_df = pd.DataFrame({
        "image_path": image_paths,
        "fold": fold_to_run,
        "Dry_Green_g": all_targets_np[:, 0],
        "Dry_Dead_g": all_targets_np[:, 1],
        "Dry_Clover_g": all_targets_np[:, 2],
        "GDM_g": all_targets_np[:, 3],
        "Dry_Total_g": all_targets_np[:, 4],
        "Dry_Green_g_pred": preds_dict_np['green'],
        "Dry_Dead_g_pred": pred_dead,
        "Dry_Clover_g_pred": pred_clover,
        "GDM_g_pred": preds_dict_np['gdm'],
        "Dry_Total_g_pred": preds_dict_np['total']
    })
    oof_dfs.append(fold_df)

    # Compute fold R²
    y_true = fold_df[["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g", "GDM_g", "Dry_Total_g"]].values
    y_pred = fold_df[["Dry_Green_g_pred", "Dry_Dead_g_pred", "Dry_Clover_g_pred", "GDM_g_pred", "Dry_Total_g_pred"]].values

    fold_r2 = r2_score(y_true, y_pred, multioutput='raw_values')
    weighted_r2 = np.dot(fold_r2, CFG.R2_WEIGHTS)
    print(f"Fold {fold_to_run} Weighted R²: {weighted_r2:.4f}")
    for name, r2 in zip(CFG.ALL_TARGET_COLS, fold_r2):
        print(f"   {name}: {r2:.4f}")

    # Cleanup
    del model, train_loader, valid_loader, train_dataset, valid_dataset
    gc.collect()
    torch.cuda.empty_cache()



# --- RUN CROSS-VALIDATION TRAINING ---
try:
    for i in range(CFG.N_FOLDS):
        run_training(fold_to_run=i)
    # ================================================================
# 🧾 COMBINE AND SAVE OOF PREDICTIONS
# ================================================================
    print("\n========== Aggregating OOF Predictions Across Folds ==========")
    
    oof_df = pd.concat(oof_dfs, ignore_index=True)
    
    # Compute overall OOF weighted R²
    y_true = oof_df[["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g", "GDM_g", "Dry_Total_g"]].values
    y_pred = oof_df[["Dry_Green_g_pred", "Dry_Dead_g_pred", "Dry_Clover_g_pred", "GDM_g_pred", "Dry_Total_g_pred"]].values
    
    r2s = r2_score(y_true, y_pred, multioutput='raw_values')
    oof_weighted_r2 = np.dot(r2s, CFG.R2_WEIGHTS)
    
    print("\n========== Overall OOF Scores ==========")
    print(f"✅ OOF Weighted R²: {oof_weighted_r2:.4f}")
    for name, r2 in zip(CFG.ALL_TARGET_COLS, r2s):
        print(f"   {name}: {r2:.4f}")
    
    # Save OOF results
    oof_df.to_csv("OOF_predictions.csv", index=False)
    print("✅ Saved OOF predictions to 'OOF_predictions.csv'")

except Exception as e:
    gc.collect()
    torch.cuda.empty_cache()
    raise e
